In [2]:
import pandas as pd

df = pd.read_csv("words.csv", header=None, names=["words"])
df = df.dropna()

df = df[~df["words"].str.isupper()]
df = df[df["words"].str.len() >= 4]
df = df[df["words"].str.match("^[a-zA-Z]+$")]

def get_filtered_words(filtered_df, input_word, condition, letter_count):
    # Apply conditions dynamically
    for i in range(letter_count):# Loop through each letter position
        #print(filtered_df.head())
        if condition[i].lower() == "g":  # If 'G', match exact position
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: x.lower()[i] == input_word[i].lower())]
    
        elif condition[i].lower() == "y":  # If 'Y', input_word[i] must be anywhere in x but **not at the same position**
            filtered_df = filtered_df[
                filtered_df['words'].apply(lambda x: input_word[i].lower() in x.lower() and x.lower()[i] != input_word[i].lower())
            ]
    
        elif condition[i].lower() == "x":  # If 'X', input_word[i] must **not** be in x at any position
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: input_word[i].lower() not in x.lower())]
    return filtered_df  # Return updated dataframe

# Function to take validated input
def get_valid_input(prompt, valid_length, valid_chars=None):
    while True:
        user_input = input(prompt).strip()  # Remove extra spaces
        if len(user_input) != valid_length:
            print(f"⚠ Error: Input must be exactly {valid_length} characters. Try again.")
            continue
        if valid_chars and any(c not in valid_chars for c in user_input):
            print(f"⚠ Error: Invalid character found! Allowed characters: {''.join(valid_chars)}. Try again.")
            continue
        return user_input  # Return valid input


letter_count = int(input("Letters: "))
chances = int(input("Chances: "))
df = df[df["words"].str.len() == letter_count]
# Reset index
df = df.reset_index(drop=True)
filtered_df = df.copy()

# Run the function 5 times with validated input
for _ in range(chances):
    user_word = get_valid_input(f"Enter a {letter_count}-letter word: ", letter_count)  # Validate word length
    user_condition = get_valid_input("Enter condition (only G, X, Y): ", letter_count, {"G", "X", "Y", "g", "x", "y"})  # Validate condition
    filtered_df = get_filtered_words(filtered_df, user_word.lower(), user_condition, letter_count)  # Update filtered_df
    print(f"Remaining words: {len(filtered_df)}")
    print(filtered_df.sample(min(len(filtered_df), 15))) 

Letters:  5
Chances:  6
Enter a 5-letter word:  cents
Enter condition (only G, X, Y):  yxxyx


Remaining words: 131
       words
14504  patch
18668  tacky
13843  octad
5607   ducat
500    aitch
19498  torch
19672  trock
5118   dicht
13069  mulct
19854  twick
19702  truck
19798  Turco
20809  watch
19222  ticca
15204  potch


Enter a 5-letter word:  tiger
Enter condition (only G, X, Y):  yxxxx


Remaining words: 32
       words
1918   batch
20611  vocat
9023   hutch
8912   hotch
13844  octal
20247  utchy
10924  Kutch
14307  Pacht
12977  Motch
13147  mutch
5722   Dutch
5607   ducat
15204  potch
12297  match
14504  patch


Enter a 5-letter word:  atoms
Enter condition (only G, X, Y):  yyxxx


...

Remaining words: 4
       words
8412   Hatch
20809  watch
11119  latch
1918   batch


Enter a 5-letter word:  batch
Enter condition (only G, X, Y):  xgggg


Remaining words: 3
       words
8412   Hatch
11119  latch
20809  watch


Enter a 5-letter word:  hatch
Enter condition (only G, X, Y):  xgggg


KeyError: 'words'

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

def load_csv():
    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if file_path:
        global df, filtered_df
        df = pd.read_csv(file_path, header=None, names=["words"])
        df = df.dropna()
        df = df[~df["words"].str.isupper()]
        df = df[df["words"].str.len() >= 4]
        df = df[df["words"].str.match("^[a-zA-Z]+$")]
        messagebox.showinfo("Success", "CSV file loaded successfully!")

def start_game():
    global filtered_df
    try:
        letter_count = int(letter_entry.get())
        chances = int(chances_entry.get())
        if letter_count < 4 or chances < 1:
            raise ValueError
        
        filtered_df = df[df["words"].str.len() == letter_count].copy()
        filtered_df.reset_index(drop=True, inplace=True)
        remaining_chances.set(chances)
        output_box.delete("1.0", tk.END)
        output_box.insert(tk.END, f"Game started! {len(filtered_df)} words loaded.\n")
    except ValueError:
        messagebox.showerror("Error", "Invalid input! Enter valid numbers for letters and chances.")

def get_filtered_words(input_word, condition):
    global filtered_df
    letter_count = len(input_word)
    for i in range(letter_count):
        if condition[i].lower() == "g":
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: x.lower()[i] == input_word[i].lower())]
        elif condition[i].lower() == "y":
            filtered_df = filtered_df[
                filtered_df['words'].apply(lambda x: input_word[i].lower() in x.lower() and x.lower()[i] != input_word[i].lower())
            ]
        elif condition[i].lower() == "x":
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: input_word[i].lower() not in x.lower())]
    return filtered_df

def apply_filter():
    global filtered_df
    if remaining_chances.get() <= 0:
        messagebox.showinfo("Game Over", "No more chances left!")
        return
    
    input_word = word_entry.get().strip()
    condition = condition_entry.get().strip()
    letter_count = int(letter_entry.get())
    
    if len(input_word) != letter_count or len(condition) != letter_count:
        messagebox.showerror("Error", "Word and condition must match the letter count!")
        return
    if not set(condition.lower()).issubset({"g", "x", "y"}):
        messagebox.showerror("Error", "Condition can only contain G, X, or Y!")
        return
    
    filtered_df = get_filtered_words(input_word.lower(), condition)
    remaining_chances.set(remaining_chances.get() - 1)
    
    output_box.delete("1.0", tk.END)
    output_box.insert(tk.END, f"Remaining words: {len(filtered_df)}\n")
    output_box.insert(tk.END, "\n".join(filtered_df['words'].sample(min(len(filtered_df), 15)).tolist()))

# GUI Setup
root = tk.Tk()
root.title("Word Filtering Game")

tk.Button(root, text="Load CSV", command=load_csv).pack(pady=5)

tk.Label(root, text="Letters:").pack()
letter_entry = tk.Entry(root)
letter_entry.pack()

tk.Label(root, text="Chances:").pack()
chances_entry = tk.Entry(root)
chances_entry.pack()

tk.Button(root, text="Start Game", command=start_game).pack(pady=5)

remaining_chances = tk.IntVar()

word_frame = tk.Frame(root)
tk.Label(word_frame, text="Word:").pack(side=tk.LEFT)
word_entry = tk.Entry(word_frame)
word_entry.pack(side=tk.LEFT)
word_frame.pack()

tk.Label(root, text="Condition (G, X, Y):").pack()
condition_entry = tk.Entry(root)
condition_entry.pack()

tk.Button(root, text="Apply Filter", command=apply_filter).pack(pady=5)

output_box = scrolledtext.ScrolledText(root, width=50, height=10)
output_box.pack()

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/s9/0x0wmn8j1rg6kj1_92zsq8pc0000gn/T/ipykernel_10223/266801387.py", line 24, in start_game
    filtered_df = df[df["words"].str.len() == letter_count].copy()
                  ^^
NameError: name 'df' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/s9/0x0wmn8j1rg6kj1_92zsq8pc0000gn/T/ipykernel_10223/266801387.py", line 24, in start_game
    filtered_df = df[df["words"].str.len() == letter_count].copy()
                  ^^
NameError: name 'df' is not defined
